In [1]:
xls.prep <- function(formula,data,dependent_var){
    
    matrix.baked <- stats::model.matrix(formula,data)
    
    df.baked <- base::as.data.frame(matrix.baked)
    
    base::rm(matrix.baked)
    
    varnum <- base::ncol(df.baked)
    
    varnames <- base::colnames(df.baked)
    
    coefnames <- base::paste0('x[',1:varnum,']')
    
    regformat <- base::paste(base::rep('%s*%s',varnum),collapse = ' + ')
    
    coef_var_match <- NULL
    
    for(i in 1:varnum){
        coef_var_match[i] <- base::sprintf('"%s",df.baked[["%s"]]',coefnames[i],varnames[i])
    }
    
    coef_var_match <- base::paste(coef_var_match,collapse = ',')
    
    symbolic_error_calculation <- base::sprintf('base::sprintf("%s",%s)',regformat,coef_var_match)
    
    base::eval(base::parse(text = base::sprintf('df.baked[["error_symbolic"]] <- %s',symbolic_error_calculation)))
    
    df.baked[["error_symbolic"]] <- base::sprintf('(%s - %s)^2',df.baked[["error_symbolic"]],data[[dependent_var]])

    base::list(data=df.baked,independent_var=varnames)

}

In [2]:
xls.objfun <- function(data,error_column_name,args){
    sum_of_errors <- base::paste(data[[error_column_name]],collapse = ' + ')
    
    base::eval(base::parse(text = base::paste('objfun <- function(', args, ') { return(' , sum_of_errors , ')}', sep='')))
    
    base::list(objective = objfun)
}

In [3]:
xls.fit <- function(formula,
                    data,
                    lag_level=1,
                    lag_column = NULL,
                    error_weights = NULL,
                    error_ahead_level=4){
    
    if(base::nrow(data) < error_ahead_level){
        
        base::stop('The number of observations must be greater than error ahead level.')
        
    }
    
    
    dependent_var <- base::all.vars(formula)[1]
    
    if(base::is.null(lag_column)){
        
        lag_column <- dependent_var
        
    }
    
    if(base::is.null(error_weights)){
        
        dummy_weights <- base::seq(from = 0,to = 1,length.out = error_ahead_level + 1)
        error_weights <- dummy_weights[-1]/base::sum(dummy_weights[-1])
        base::rm(dummy_weights)
        
    }else if(base::length(error_weights) != error_ahead_level){
        
        base::stop('Error weights should have same length with ahead level.')
        
    }else if(base::sum(error_weights) != 1){
        
        base::stop('The sum of the error weights must be 1.')
        
    }
    
    prepared_obj <- xls.prep(formula,data,dependent_var)
    
    df <- prepared_obj$data
    
    independent_var <- prepared_obj$independent_var
    
    initial_solution <- base::rep(0,base::length(independent_var))

    objfun_object <- xls.objfun(df,'error_symbolic',args = 'x')
    
    objfun <- objfun_object$objective
    
    base::suppressWarnings(optimizing_parameters <- NlcOptim::solnl(X = initial_solution,objfun = objfun))
    
    coefficients <- base::as.data.frame(optimizing_parameters$par)
    
    base::colnames(coefficients) <- 'coef'
    
    coefficients[['term']] <- independent_var
    
    coefficients

}

In [4]:
as.data.frame(coef(lm(Sepal.Width ~ Petal.Length:Species + Petal.Width - 1,iris)))

,"coef(lm(Sepal.Width ~ Petal.Length:Species + Petal.Width - 1, iris))"
,<dbl>
Petal.Width,0.6891438
Petal.Length:Speciessetosa,2.2017762
Petal.Length:Speciesversicolor,0.4323144
Petal.Length:Speciesvirginica,0.2826475


In [5]:
xls.fit(Sepal.Width ~ Petal.Length:Species + Petal.Width - 1,iris)

coef,term
<dbl>,<chr>
0.6891456,Petal.Width
2.2017759,Petal.Length:Speciessetosa
0.4323138,Petal.Length:Speciesversicolor
0.2826468,Petal.Length:Speciesvirginica
